In [25]:
# imports 

from bs4 import BeautifulSoup
import requests
from urllib.parse import urljoin
import json


In [26]:
## original url for the rosters
base_url = "https://fightingillini.com/sports/mens-basketball/roster"
res = requests.get(base_url)
soup = BeautifulSoup(res.text, "html.parser")
# this is the tag with the dropdown menu for all the rosters
select_tag = soup.find("select", id="ddl_past_rosters")
options = select_tag.find_all("option") if select_tag else []

season_dict = {}
player_dict = {}
no_names = []
base_url = "https://fightingillini.com"
count = 0
# for option in rosters
for option in options:
    # ex. 2024-25
    season_name = option.get_text(strip=True)[0:7]
    # ex. /sports/mens-basketball/roster/2024-25
    relative_url = option.get("value")
    ## if i need a dictionary of the roster urls, do it here 
    
    if season_name:
        full_url = urljoin(base_url, relative_url)
    season_dict[season_name] = base_url + "/sports/mens-basketball/stats/" + season_name
    res = requests.get(full_url)
    soup = BeautifulSoup(res.text, "html.parser")    

    
    count += 1
    for li in soup.find_all("li", class_="sidearm-roster-player"):
    # Get the profile URL from the data attribute
        relative_url = li.get("data-player-url")
        if relative_url:
            full_url = urljoin(base_url, relative_url)
        else:
         full_url = None

    # Find the <a> tag that has the player's name (not "Full Bio")
        name_tag = li.select_one("h3 > a")
        if not name_tag or not name_tag.text.strip():
            no_names.append(full_url)
            continue
        if name_tag and full_url:
            player_name = name_tag.text.strip()
            # I needed to check if theres already a year, cause then I'll just append instead of add a new person
            if (player_name in player_dict) and (len(player_dict[player_name][1]) > 0):
                if '26' not in season_name:
                    player_dict[player_name][1].append(season_name)
            else:
                if '26' not in season_name:
                    player_dict[player_name] = [full_url, [season_name]]


            
    


In [27]:
player_dict

{'Carey Booth': ['https://fightingillini.com/sports/mens-basketball/roster/carey-booth/14522',
  ['2024-25']],
 'Dra Gibbs-Lawhorn': ['https://fightingillini.com/sports/mens-basketball/roster/dra-gibbs-lawhorn/14512',
  ['2024-25', '2023-24']],
 'Ben Humrichous': ['https://fightingillini.com/sports/mens-basketball/roster/ben-humrichous/14519',
  ['2024-25']],
 'Kylan Boswell': ['https://fightingillini.com/sports/mens-basketball/roster/kylan-boswell/14523',
  ['2024-25']],
 'AJ Redd': ['https://fightingillini.com/sports/mens-basketball/roster/aj-redd/14514',
  ['2024-25', '2023-24', '2022-23']],
 'Will Riley': ['https://fightingillini.com/sports/mens-basketball/roster/will-riley/14574',
  ['2024-25']],
 'Tomislav Ivisic': ['https://fightingillini.com/sports/mens-basketball/roster/tomislav-ivisic/14520',
  ['2024-25']],
 'Jake Davis': ['https://fightingillini.com/sports/mens-basketball/roster/jake-davis/14518',
  ['2024-25']],
 'Ty Rodgers': ['https://fightingillini.com/sports/mens-baske

In [28]:
print(player_dict["Carey Booth"])

['https://fightingillini.com/sports/mens-basketball/roster/carey-booth/14522', ['2024-25']]


In [29]:
print(player_dict["Chester Frazier"])

['https://fightingillini.com/sports/mens-basketball/roster/chester-frazier/3332', ['2008-09', '2007-08', '2006-07', '2005-06']]


In [ ]:
with open('player_list.json', 'w') as f:
    json.dump(player_dict, f, indent=4)

FileNotFoundError: [Errno 2] No such file or directory: '/app/player_list.json'

In [36]:
print(player_dict["Ben Humrichous"])

['https://fightingillini.com/sports/mens-basketball/roster/ben-humrichous/14519', ['2024-25']]
